In [4]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum
from scipy.integrate import quad

yu.setpath('lhapdf')

if "_has_run_once" not in globals():
    import os, sys, ctypes
    _has_run_once = True
    path='/p/project1/ngff/li47/code/LHAPDF/build/lib/'
    # path='/p/project1/ngff/li47/code/temp/LHAPDF/build/lib/'
    sys.path.insert(0, f'{path}python3.13/site-packages')
    ctypes.CDLL(f'{path}libLHAPDF.so', mode=ctypes.RTLD_GLOBAL)
    
    'python3.13/site-packages'
    import lhapdf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
lhapdf.availablePDFSets()

['NNPDF31_nnlo_as_0118',
 'NNPDF40_an3lo_as_01180',
 'NNPDF40_nlo_as_01180',
 'NNPDF40_nnlo_as_01180',
 'NNPDF40_nnlo_as_01180_qed']

In [6]:
PDF_SET      = "NNPDF40_nnlo_as_01180"
pdfset = lhapdf.getPDFSet(PDF_SET)
replicas = pdfset.mkPDFs()

LHAPDF 6.5.5 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members


In [28]:
# avgx
jqs=['ju','jd','js','jc']
def j2cinds(j):
    if j=='jtot':
        return j2cinds('jq')+j2cinds('jg')
    if j=='jv1':
        t=j2cinds('jd')
        t=[(-c,ind) for c,ind in t]
        return j2cinds('ju')+t
    if j in ['jg']:
        return [(1,21)]
    if j in jqs:
        ind=[2,1,3,4][jqs.index(j)]
        return [(1,ind),(1,-ind)]
    if j[:2]=='jq':
        return [cind for jq in jqs for cind in j2cinds(jq+j[2:])]

def pdf_moment(pdf, cinds, n=1):
    Q=2.0
    func = lambda x: x**(n-1) * sum(c*pdf.xfxQ(ind, x, Q) for c,ind in cinds)
    val, _ = quad(func, 1e-6, 1.0, epsabs=1e-6, epsrel=1e-4)
    return val
    
js=['jtot','jq','jg','ju','jd','js','jc','jv1']
j2dat={}
for j in js:
    dat=[pdf_moment(pdf, j2cinds(j)) for pdf in replicas]
    j2dat[j]=np.array(dat)

j2me={}
for j in js:
    dat=j2dat[j]
    m,e=dat.mean(),dat.std(ddof=1)
    j2me[j]=(m,e)
    print(j,yu.un2str(m,e))
    if j in ['jg','jc']:
        print()
        
yu.save_pkl_reg('j2me_NNPDF40_nnlo_as_01180',j2me)

jtot 0.9999969(47)
jq 0.6005(26)
jg 0.3995(26)

ju 0.3468(26)
jd 0.1934(32)
js 0.0483(34)
jc 0.0119(22)

jv1 0.1533(41)
